# Исследование надежности заемщиков.

**Заказчик** — кредитный отдел банка

**Цель Заказчика.** Снижение рисков по кредитным операциям посредством настройки системы оценки способности потенциального заёмщика своевременно возвращать кредит банку.

**Цель исследования.** Для построения модели кредитного скоринга определить влияние семейного положения и количества детей клиента на факт погашения им кредита в установленный срок

**Задачи**
* Определить наличие зависимости между количеством детей и возвратом кредита в срок
* Определить наличие зависимости между семейным положением и возвратом кредита в срок
* Определить наличие зависимости между уровнем дохода и возвратом кредита в срок
* Выявить влияние целей кредита на своевременность его возврата

**Входные данные от Заказчика.** Файл в формате .csv со статистикой о платёжеспособности клиентов.

**Ожидаемый результат.** Определены факторы, коррелирующие со способностью заёмщика своевременно вернуть кредит банку

### Шаг 1. Обзор данных

1. Открыть таблицу 
2. Изучить структуру данных по десяти первым строкам
3. Изучить структуру данных методом info()

**Описание данных**
- **children** — количество детей в семье
- **days_employed** — общий трудовой стаж в днях
- **dob_years** — возраст клиента в годах
- **education** — уровень образования клиента
- **education_id** — идентификатор уровня образования
- **family_status** — семейное положение
- **family_status_id** — идентификатор семейного положения
- **gender** — пол клиента
- **income_type** — тип занятости
- **debt** — имел ли задолженность по возврату кредитов
- **total_income** — ежемесячный доход
- **purpose** — цель получения кредита

In [1]:
#Импорт библиотеки
import pandas as pd

In [2]:
#Чтение файла
df = pd.read_csv('/datasets/data.csv')

In [3]:
#Выгрузка первых десяти строк таблицы 
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
#Получение общей информации по таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Общие выводы по качеству имеющихся данных
1. Таблица содержит 12 столбцов и 21 525 строк
2. Заголовки столбцов корректны и отражают суть данных
3. Столбцы "days_employed" и "total_income" содержат пропуски
4. В столбцах "days_employed", "family_status", "total_income" не корректный тип данных
5. В столбце "education" встречаются неявные дубли. Есть вероятность повторения этой ситуации в других столбцах.

### Шаг 2.1 Заполнение пропусков

По информации из таблицы пропуски есть в столбцах days_employed и total_income. Пропуски имеют тип NaN. Количество пропусков одинаковое и составляет 10% от общего числа значений. Сопоставление значений в столбце "total_income" при наличии пропуска в столбце "days_employed" показало, что пропуски присутствуют только в обоих столбцах одновременно. Это говорит о системной ошибке заполнения данных.

Группировка по типам занятости показала, что каждый из типов значительно отличается от другого по средним значениям трудового стажа и уровня дохода. В тоже время, есть значительное отклонение медианных от средних значений, что говорит о наличии "артефактов" и "выбросов".

Таким образом, наименьшее снижение точности дальнейшего исследования обеспечит заполнение пропусков медианными значениями для каждой группы занятости.

In [5]:
# Проверка на общее количество пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# Выявление доли пропусков по столбцам
total_str = int(df.shape[0])
null_days_employed = int(df['days_employed'].isna().sum())
null_total_income = int(df['total_income'].isna().sum())

print(f'Всего в дадафрейме {total_str} строк')
print(f'В столбце days_employed пропущено {null_days_employed} строк, что составляет {null_days_employed / total_str:.0%}')
print(f'В столбце total_income пропущено {null_total_income} строк, что составляет {null_total_income / total_str:.0%}')

Всего в дадафрейме 21525 строк
В столбце days_employed пропущено 2174 строк, что составляет 10%
В столбце total_income пропущено 2174 строк, что составляет 10%


In [7]:
# Сопоставление значений в столбце "уровень дохода" при наличии пропуска в столбце "трудовой стаж"
print(df[df['days_employed'].isna() == True]['total_income'].unique())

[nan]


In [8]:
# Определение средних и медианных значений для различных групп занятости
display(df.groupby('income_type').agg({'days_employed':['mean', 'median'], 'total_income':['mean', 'median']}))

days_employed                  total_income               
                          mean         median           mean         median
income_type                                                                
безработный      366413.652744  366413.652744  131339.751676  131339.751676
в декрете         -3296.759962   -3296.759962   53829.130729   53829.130729
госслужащий       -3399.896902   -2689.368353  170898.309923  150447.935283
компаньон         -2111.524398   -1547.382223  202417.461462  172357.950966
пенсионер        365003.491245  365213.306266  137127.465690  118514.486412
предприниматель    -520.848083    -520.848083  499163.144947  499163.144947
сотрудник         -2326.499216   -1574.202821  161380.260488  142594.396847
студент            -578.751554    -578.751554   98201.625314   98201.625314

In [9]:
# Замена пропусков на медианные значение по каждлй группе занятости
# Создвем список групп занятости
list_income_type = df['income_type'].unique()
median_days_employed = []
median_total_income = []

# По списку групп занятости заполняем медианные значения трудового стажа и средней зарплаты
for lists in list_income_type:
    median_days_employed.append(df[df['income_type'] == lists]['days_employed'].median())
    median_total_income.append(df[df['income_type'] == lists]['total_income'].median())

# Заменяем пропуски на медианнве значения по каждой группы звнятости для каждого из столбцов
for i in range(len(list_income_type)):
    df.loc[(df['days_employed'].isna() == True)&(df['income_type'] == list_income_type[i]), 'days_employed'] = median_days_employed[i]
    df.loc[(df['total_income'].isna() == True)&(df['income_type'] == list_income_type[i]), 'total_income'] = median_total_income[i]


In [10]:
# Проверим наличие пропусков в таблице
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

В столбце `days_employed` встречаются артефакты (аномалии) — значения, которые не отражают действительность и появились в результате ошибки. 
Речь идет об отрицательных значениях количества дней трудового стажа. Кроме того, значения для пенсионеров и безработных явно выбиваются из значений других категорий.

Такая ошибки скорее всего возникли либо в результате получения данных из разных систем, либо в результате различных форматов получаемых данных.

Учитывая то, понять логику формирования значений по трудовому стажу для пенсионеров и безработных не представляется возможным, **ограничимся только заменой отрицательных значений на положительные**.

In [11]:
# Функция для замены отрицательного числа на полодительное
def to_plus(x):
    if x < 0:
        x *= -1
    return x

# Замена отрицательных значений на положительные
df["days_employed"] = df["days_employed"].apply(to_plus)

# Проверка корректности проведенной замены
display(df.groupby('income_type').agg({'days_employed':['mean', 'median']}))

days_employed               
                          mean         median
income_type                                  
безработный      366413.652744  366413.652744
в декрете          3296.759962    3296.759962
госслужащий        3328.308350    2689.368353
компаньон          2055.165652    1547.382223
пенсионер        365025.963652  365213.306266
предприниматель     520.848083     520.848083
сотрудник          2251.736421    1574.202821
студент             578.751554     578.751554

### Шаг 2.3. Изменение типов данных.

В столбце `total_income` заменим вещественный тип данных на целочисленный с помощью метода `astype()`

In [12]:
# Проверка типа данных в столбце total_income
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [13]:
# Замена вещественного типа данных на целочисленный с помощью метода astype()
df['total_income'] = df['total_income'].astype('int')

In [14]:
# Повторая проверка типа данных в столбце total_income
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

1. Удалить строки-дубликаты методом `drop_duplicates()`

In [15]:
#Проверка наличия дубликатов в таблице
df.duplicated().sum()

54

In [16]:
# Удаление явных дубликатов
df = df.drop_duplicates().reset_index(drop=True)

In [17]:
#Повторная проверка наличия дубликатов в таблице
df.duplicated().sum()

0

2. Обработать неявные дубликаты. 

При первом просмотре таблицы стало ясно, что в столбце `education` есть одни и те же значения, но записанные с использованием заглавных и строчных букв. Эти названия будут приведены в одному регистру. После этого будут проверены остальные столбцы на наличие дубликатов методом `unique()`.

In [18]:
# Напишем функцию для смены регистра и применим ее к столбцу education, family_status, 
def to_lower(x):
    return x.lower()
df["education"] = df["education"].apply(to_lower)

In [19]:
# Проверим наличие дублей в стобцах: education, gender, income_type
list_columns = ['children', 'dob_years', 'education', 'gender', 'income_type', 'debt']
for list_column in list_columns:
    print(df[list_column].sort_values().unique())

[-1  0  1  2  3  4  5 20]
[ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]
['высшее' 'начальное' 'неоконченное высшее' 'среднее' 'ученая степень']
['F' 'M' 'XNA']
['безработный' 'в декрете' 'госслужащий' 'компаньон' 'пенсионер'
 'предприниматель' 'сотрудник' 'студент']
[0 1]


Среди исследованных столбцов, столбцы `children` и `gender` содержат неопределенное значения: `-1` и `XNA` соответственно

Вероятно, это ошибочно введенные данные. Значение `XNA` принадлежит одной строке.

Поскольку определить половую принадлежность заемщика не представляется возможным, то данная строка должна быть удалена

Значение `-1` принадлежит 47 строкам. Поэтому будет эффективно заменить его на `0`

In [20]:
df['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

In [21]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10690,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


In [22]:
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [23]:
# Заменить количество детей с -1 на 0
df['children'] = df['children'].replace(-1, 0)

In [24]:
df['children'].value_counts()

0     14154
1      4809
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [25]:
# Удаление строки с признаком XNA
df = df[df['gender'] != 'XNA'].reset_index(drop=True)

In [26]:
# Контрольная проверка отсутствия строки с признаком XNA
df['gender'].value_counts()

F    14189
M     7281
Name: gender, dtype: int64

**Дубликаты удалены.** 
* Для поиска дубликатов был выбран метод `unique()`
* Для удаления дубликатов был применен метод `apply()`
* Причина появление неявных дубликатов скорее всего возникла в результате ручного ввода не категорированных данных. Это системная ошибка.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

1. Создать два новых датафрейма со столбцами:
 * в первом датафрейме `df_education` - `education_id` и `education`;
 * во втором датафрейме `df_family` - `family_status_id` и `family_status`.


2. Удалить из исходного датафрейма столбцы `education` и `family_status`

In [27]:
# Создание датафрейма df_education

education_id_unique = df['education_id'].unique()
education_unique = []

for i in education_id_unique:
    education_unique.append(list(df[df['education_id'] == i]['education'])[0])

df_education = pd.DataFrame({'education_id': education_id_unique, 'education': education_unique})

display(df_education)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [28]:
# Создание датафрейма df_family

family_status_id_unique = df['family_status_id'].sort_values().unique()
family_status_unique = []

for i in family_status_id_unique:
    family_status_unique.append(list(df[df['family_status_id'] == i]['family_status'])[0])

df_family = pd.DataFrame({'family_status_id': family_status_id_unique, 'family_status': family_status_unique})

display(df_family)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [29]:
# Удаляем из исходного датафрейма столбцы education и family_status
df = df.drop(['education', 'family_status'], axis=1)
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

**Cоздать новый столбец с категориями** `total_income_category` на основании указанных диаразонов:

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [30]:
# Создадим функцию для формирвания категорий
def total_income_category(sum_incom):
    if sum_incom <= 30000:
        return 'E'
    elif sum_incom <= 50000:
        return 'D'
    elif sum_incom <= 200000:
        return 'C'
    elif sum_incom <= 1000000:
        return 'B'
    return 'A'

# Создадим новый столбец
df['total_income_category'] = df['total_income'].apply(total_income_category)

### Шаг 2.7. Категоризация целей кредита.

**Создать новый столбец** `purpose_category` с указанными ниже категориями:

* ‘операции с автомобилем’,
* ‘операции с недвижимостью’,
* ‘проведение свадьбы’,
* ‘получение образования’.

In [31]:
# Создадим функцию для формирвания категорий
def purpose_category(categor):
    if 'автомоб' in categor:
        return 'операции с автомобилем'
    elif 'жиль' in categor or 'недвиж' in categor:
        return 'операции с недвижимостью'
    elif 'свадьб' in categor:
        return 'проведение свадьбы'
    elif 'образован' in categor:
        return 'получение образования'

# Создадим новый столбец
df['purpose_category'] = df['purpose'].apply(purpose_category)   

### Категоризация возраста.

**Создать новый столбец с категориями** `dob_years_category` на основании указанных диапазонов:

* 'до 20 лет';
* 'от 21 до 40 лет';
* 'от 41 до 60 лет';
* 'старше 61 лет'.

In [32]:
# Создадим функцию для формирвания категорий
def dob_years_category(agg):
    if agg <= 20:
        return 'до 20 лет'
    elif agg <= 40:
        return 'от 21 до 40 лет'
    elif agg <= 60:
        return 'от 41 до 60 лет'
    return 'старше 61 лет'

# Создадим новый столбец
df['dob_years_category'] = df['dob_years'].apply(dob_years_category)

In [33]:
# Проверим наличие новых столбцов
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,dob_years_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,от 41 до 60 лет
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,от 21 до 40 лет
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,от 21 до 40 лет
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,от 21 до 40 лет
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,от 41 до 60 лет


### Выводы по разделу предобработка данных

В рамках подготовки датафрейма были выявлены и заполнены медианными значениями пропуски. Изменены (обработаны) аномальные значения. В столбцах установлен корректный тип данных. Удалены как явные, так и не явные дубликаты. Созданы новые таблицы-словари, что позволило убрать из текущей таблицы лишние столбцы, а также добавлены столбцы с новыми категориями.

### Ответы на вопросы.

В рамках решаемых задач необходимо раскрыть следующие вопросы:
* Определить наличие зависимости между количеством детей и возвратом кредита в срок
* Определить наличие зависимости между семейным положением и возвратом кредита в срок
* Определить наличие зависимости между уровнем дохода и возвратом кредита в срок
* Выявить влияние целей кредита на своевременность его возврата

##### Вопрос 1:
**Определение  наличия или отсутствия зависимости между количеством детей и возвратом кредита в срок**

In [34]:
# Создадим сводную таблицу для определения влияния количества детей 
# в семье на возвращаемость кредита. Последний столбец – доля возвращаемости

df_child = df.pivot_table(index='children', columns='debt', values='income_type', aggfunc='count')
df_child[2] = df_child[0] / (df_child[0] + df_child[1])
display(df_child)

debt,0,1,2
children,,,
0,13089.0,1064.0,0.924822
1,4365.0,444.0,0.907673
2,1858.0,194.0,0.905458
3,303.0,27.0,0.918182
4,37.0,4.0,0.902439
5,9.0,NaN,NaN
20,68.0,8.0,0.894737


##### Вывод 1:
Люди, не имеющие детей на 2% чаще возвращают кредит, чем люди с детьми

##### Вопрос 2:
Определение наличия зависимости между семейным положением и возвратом кредита в срок

In [35]:
# Создадим сводную таблицу для определения влияния семейного положения 
# на возвращаемость кредита. Последний столбец – доля возвращаемости

df_fam = df.pivot_table(index='family_status_id', columns='debt', values='income_type', aggfunc='count')
df_fam[2] = df_fam[0] / (df_fam[0] + df_fam[1])

# Добавим столбец с описанием семейного статуса из справочной таблицы df_family

df_fam = df_family.merge(df_fam, on='family_status_id', how='left')
display(df_fam)

,family_status_id,family_status,0,1,2
0,0,женат / замужем,11413,931,0.924579
1,1,гражданский брак,3774,388,0.906776
2,2,вдовец / вдова,896,63,0.934307
3,3,в разводе,1110,85,0.928870
4,4,Не женат / не замужем,2536,274,0.902491


##### Вывод 2:
Женатые и разведенные на 2% чаще возвращают кредит, чем незамужние, вдовцы или люди, находящиеся в гражданском браке

##### Вопрос 3:
Определение наличия зависимости между уровнем дохода и возвратом кредита в срок

In [36]:
# Создадим сводную таблицу для определения влияния уровня дохода 
# на возвращаемость кредита. Последний столбец – доля возвращаемости
df_rev = df.pivot_table(index='total_income_category', columns='debt', values='income_type', aggfunc='count')
df_rev[2] = df_rev[0] / (df_rev[0] + df_rev[1])
display(df_rev)

debt,0,1,2
total_income_category,,,
A,23,2,0.920000
B,4685,356,0.929379
C,14672,1360,0.915170
D,329,21,0.940000
E,20,2,0.909091


##### Вывод 3:
Люди с доходом от 30 000 р. до 50 000 р. (нижний уровень) 2% чаще возвращают кредит, чем люди с более высоким уровнем дохода

##### Вопрос 4:
Выявление влияния целей кредита на своевременность его возврата

In [37]:
# Создадим сводную таблицу для определения влияния целей кредита 
# на его возвращаемость. Последний столбец – доля возвращаемости

df_purpose = df.pivot_table(index='purpose_category', columns='debt', values='income_type', aggfunc='count')
df_purpose[2] = df_purpose[0] / (df_purpose[0] + df_purpose[1])
display(df_purpose)

debt,0,1,2
purpose_category,,,
операции с автомобилем,3905,403,0.906453
операции с недвижимостью,10031,782,0.927680
получение образования,3644,370,0.907823
проведение свадьбы,2149,186,0.920343


##### Вывод 4:
Люди, взявшие кредит на операции с недвижимостью и на проведение свадьбы, возвращают его на 2% чаще, чем люди, взявшие его на операции с автомобилем или на получение образования

##### Дополнительные метрики
1. Влияние типа занятости
2. Влияние возраста
3. Влияние пола

In [38]:
df_incom = df.pivot_table(index='income_type', columns='debt', values='dob_years', aggfunc='count')
df_incom[2] = df_incom[0] / (df_incom[0] + df_incom[1])
display(df_incom)

debt,0,1,2
income_type,,,
безработный,1.0,1.0,0.500000
в декрете,NaN,1.0,NaN
госслужащий,1371.0,86.0,0.940975
компаньон,4703.0,376.0,0.925970
пенсионер,3621.0,216.0,0.943706
предприниматель,2.0,NaN,NaN
сотрудник,10030.0,1061.0,0.904337
студент,1.0,NaN,NaN


In [39]:
df_agg = df.pivot_table(index='dob_years_category', columns='debt', values='income_type', aggfunc='count')
df_agg[2] = df_agg[0] / (df_agg[0] + df_agg[1])
display(df_agg)

debt,0,1,2
dob_years_category,,,
до 20 лет,153,13,0.921687
от 21 до 40 лет,8441,944,0.899414
от 41 до 60 лет,9104,683,0.930214
старше 61 лет,2031,101,0.952627


In [40]:
df_gend = df.pivot_table(index='gender', columns='debt', values='income_type', aggfunc='count')
df_gend[2] = df_gend[0] / (df_gend[0] + df_gend[1])
display(df_gend)

debt,0,1,2
gender,,,
F,13195,994,0.929946
M,6534,747,0.897404


## Общий вывод:

Исследование не выявило явных зависимостей между вводными параметрами и степенью возвращаемости кредитов:
* Люди, не имеющие детей на 2% чаще возвращают кредит, чем люди с детьми
* Женатые и разведенные на 2% чаще возвращают кредит, чем незамужние, вдовцы или люди, находящиеся в гражданском браке
* Люди с доходом от 30 000 р. до 50 000 р. (нижний уровень) 2% чаще возвращают кредит, чем люди с более высоким уровнем дохода
* Люди, взявшие кредит на операции с недвижимостью и на проведение свадьбы, возвращают его на 2% чаще, чем люди, взявшие его на операции с автомобилем или на получение образования

Тем не менее, можно утверждать, что заемщик женатый или разведенный клиент, не имеющий детей с доходом до 50 000 р, планирующий операции с недвижимостью имеет более высокую вероятность возврата кредита, чем какой-либо иной клиент

Дополнительные разрезы показали, что наибольший процент возврата по типу занятости – у госслужащих и у пенсионеров. Женщины возвращают на 3% чаще мужчин. Среди возрастных категорий самая высокая возворащаемость у людей старше 60 лет